In [1]:
from joblib import Parallel, delayed

import imageio
import skvideo.io as skv
from moviepy.editor import *

import numpy as np
import pandas as pd

In [2]:
data_dir = 'data/'
data_micro_dir = data_dir+'micro/'

train_labels_df = pd.read_csv(data_dir + 'train_labels.csv', index_col='filename')

In [3]:
def imageio_load_video(folder, file_name, dtype=np.int8):
    reader = imageio.get_reader(folder + file_name)
    video = np.array([frame for frame in reader.iter_data()], dtype=dtype)
    reader.close()
    return video

def moviepy_load_video(folder, file_name, dtype=np.int8):
    clip = VideoFileClip(folder + file_name, audio=False)
    return np.array([frame for frame in clip.iter_frames()], dtype=dtype)

def skv_load_video(folder, file_name, dtype=np.int8):
    return skv.vread(folder + file_name)

def skv_gen_load_video(folder, file_name, dtype=np.int8):
    videogen = skv.vreader(folder + file_name)
    return np.array([frame for frame in videogen])

def load_videos(folder, file_names, load_fun, dtype=np.int8, cpu_cores=8):
    return np.array(Parallel(n_jobs=cpu_cores)(delayed(load_fun)(folder, file_name, dtype) for file_name in file_names), dtype=dtype)

In [4]:
videos_fnames = train_labels_df.index[:1000]

In [5]:
# baseline
%time videos = np.array([skv.vread(data_micro_dir + video_fname) for video_fname in videos_fnames], dtype=np.int8)

print(videos.shape)
del videos

CPU times: user 1.19 s, sys: 7.75 s, total: 8.94 s
Wall time: 1min 14s
(1000, 30, 64, 64, 3)


In [6]:
# imageio
%time videos = load_videos(data_micro_dir, videos_fnames, imageio_load_video)

print(videos.shape)
del videos

CPU times: user 536 ms, sys: 264 ms, total: 800 ms
Wall time: 23 s
(1000, 30, 64, 64, 3)


In [7]:
# moviepy
%time videos = load_videos(data_micro_dir, videos_fnames, moviepy_load_video)

print(videos.shape)
del videos

CPU times: user 392 ms, sys: 336 ms, total: 728 ms
Wall time: 16.8 s
(1000, 30, 64, 64, 3)


In [8]:
# skv
%time videos = load_videos(data_micro_dir, videos_fnames, skv_load_video)

print(videos.shape)
del videos

CPU times: user 356 ms, sys: 364 ms, total: 720 ms
Wall time: 17.5 s
(1000, 30, 64, 64, 3)


In [9]:
# skv
%time videos = load_videos(data_micro_dir, videos_fnames, skv_gen_load_video)

print(videos.shape)
del videos

CPU times: user 364 ms, sys: 372 ms, total: 736 ms
Wall time: 17.6 s
(1000, 30, 64, 64, 3)
